In [2]:
%pip install -q llama-index deeplake openai cohere dotenv
%pip install llama-index-vector-stores-deeplake llama-index-llms-openai
%pip install llama-index-agent-openai llama-index-agent-openai-legacy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-agent-openai 0.4.12 requires llama-index-core<0.13,>=0.12.41, but you have llama-index-core 0.13.5 which is incompatible.
llama-index-agent-openai 0.4.12 requires llama-index-llms-openai<0.5,>=0.4.0, but you have llama-index-llms-openai 0.5.4 which is incompatible.
llama-index-agent-openai-legacy 0.3.4 requires llama-index-core<0.13,>=0.12.41, but you have llama-index-core 0.13.5 which is incompatible.
llama-index-agent-openai-legacy 0.3.4 requires llama-index-llms-openai<0.5,>=0.4.0, but you have llama-index-llms-openai 0.5.4 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run:

In [3]:
import os
from dotenv import load_dotenv


load_dotenv("../.env")
assert os.getenv("OPENAI_API_KEY")
assert os.getenv("ACTIVELOOP_TOKEN")

# Prepare Indexes

In [5]:
!mkdir -p "data/1k/"
!curl "https://github.com/idontcalculate/data-repo/blob/main/machine_to_end_war.txt" -o "data/1k/tesla.txt"
!curl "https://github.com/idontcalculate/data-repo/blob/main/prodigal_chapter10.txt" -o "data/1k/web.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  195k    0  195k    0     0   283k      0 --:--:-- --:--:-- --:--:--  283k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  205k    0  205k    0     0   268k      0 --:--:-- --:--:-- --:--:--  268k


### From VectorStore

In [4]:
from llama_index.core import SimpleDirectoryReader


tesla_docs = SimpleDirectoryReader( input_files=["data/1k/tesla.txt"] ).load_data()

In [5]:
from llama_index.vector_stores.deeplake import DeepLakeVectorStore


my_activeloop_org_id = "yaroslava"
my_activeloop_dataset_name = "LlamaIndex_tesla_predictions"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)

2025-09-05 16:34:27,828 - WARNING - [S3] Failed to get bucket region for URL: snark-hub/protected/yaroslava/LlamaIndex_tesla_predictions/ with error: [S3] INVALID_ACCESS_KEY_ID snark-hub The AWS Access Key Id you provided does not exist in our records. 


In [6]:
from llama_index.core import StorageContext


storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [7]:
from llama_index.core import VectorStoreIndex


tesla_index = VectorStoreIndex.from_documents(tesla_docs, storage_context=storage_context)

2025-09-05 16:34:41,634 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## From Local Index

In [8]:
webtext_docs = SimpleDirectoryReader(input_files=["data/1k/web.txt"] ).load_data()

In [9]:
from llama_index.core import StorageContext, load_index_from_storage


try:
  # Try to load the index if it is already calculated
  storage_context = StorageContext.from_defaults( persist_dir="data/storage/webtext" )
  webtext_index = load_index_from_storage(storage_context)
  print("Loaded the pre-computed index.")
except:
  # Otherwise, generate the indexes
  webtext_index = VectorStoreIndex.from_documents(webtext_docs)
  webtext_index.storage_context.persist(persist_dir="data/storage/webtext")
  print("Generated the index.")

Loading llama_index.core.storage.kvstore.simple_kvstore from data/storage/webtext/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from data/storage/webtext/index_store.json.


2025-09-05 16:34:49,074 - INFO - Loading all indices.


Loaded the pre-computed index.


# Create Query Enginges

In [10]:
tesla_engine = tesla_index.as_query_engine(similarity_top_k=3)
webtext_engine = webtext_index.as_query_engine(similarity_top_k=3)

# Create the Tools

In [11]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata


query_engine_tools = [
    QueryEngineTool(
        query_engine=tesla_engine,
        metadata=ToolMetadata(
            name="tesla_1k",
            description=(
                "Provides information about Tesla's statements that refers to future times and predictions. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=webtext_engine,
        metadata=ToolMetadata(
            name="webtext_1k",
            description=(
                "Provides information about tesla's life and biographical data. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

# Define the Agent

In [12]:
from llama_index.agent.openai import OpenAIAgent


agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True)

/Users/coder/Desktop/dev/rag_for_production_activeloop/rag-for-production-with-langchain-and-llamaindex/.venv/lib/python3.12/site-packages/llama_index/agent/openai/base.py:144: DeprecationWarning: Call to deprecated class OpenAIAgent. (OpenAIAgent has been deprecated and is not maintained.

`FunctionAgent` is the recommended replacement.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
/Users/coder/Desktop/dev/rag_for_production_activeloop/rag-for-production-with-langchain-and-llamaindex/.venv/lib/python3.12/site-packages/deprecated/classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)
/Users/coder/Desktop/dev/rag_for_produc

In [13]:
# agent.chat_repl()
response = agent.chat("What influenced Nikola Tesla to become an inventor?")
response

Added user message to memory: What influenced Nikola Tesla to become an inventor?


2025-09-05 16:35:02,112 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: webtext_1k with args: {"input":"What influenced Nikola Tesla to become an inventor?"}


2025-09-05 16:35:02,422 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-05 16:35:03,158 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Got output: Nikola Tesla was influenced to become an inventor by his interest in the effects of mechanical vibrations and their potential applications.



2025-09-05 16:35:04,057 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AgentChatResponse(response='Nikola Tesla was influenced to become an inventor by his interest in the effects of mechanical vibrations and their potential applications.', sources=[ToolOutput(blocks=[TextBlock(block_type='text', text='Nikola Tesla was influenced to become an inventor by his interest in the effects of mechanical vibrations and their potential applications.')], tool_name='webtext_1k', raw_input={'input': 'What influenced Nikola Tesla to become an inventor?'}, raw_output=Response(response='Nikola Tesla was influenced to become an inventor by his interest in the effects of mechanical vibrations and their potential applications.', source_nodes=[NodeWithScore(node=TextNode(id_='757983b3-0b15-4da6-a916-d067e025ffb1', embedding=None, metadata={'file_path': 'data/1k/web.txt', 'file_name': 'web.txt', 'file_type': 'text/plain', 'file_size': 210125, 'creation_date': '2025-09-05', 'last_modified_date': '2025-09-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size'

# Agents with Tools

In [14]:
from llama_index.core.tools import FunctionTool


def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


multiply_tool = FunctionTool.from_defaults(fn=multiply, name="multiply")
add_tool = FunctionTool.from_defaults(fn=add, name="add")

all_tools = [multiply_tool, add_tool]

In [15]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex, SimpleToolNodeMapping


tool_mapping = SimpleToolNodeMapping.from_objects(all_tools)
obj_index = ObjectIndex.from_objects(
    all_tools,
    tool_mapping,
    VectorStoreIndex,
)

2025-09-05 16:35:08,007 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [17]:
from llama_index.agent.openai_legacy import FnRetrieverOpenAIAgent


agent = FnRetrieverOpenAIAgent.from_retriever(
    obj_index.as_retriever(), verbose=True
)

In [18]:
agent.chat("What's 12 multiplied by 22? Make sure to use Tools")

2025-09-05 16:35:22,906 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


STARTING TURN 1
---------------



2025-09-05 16:35:23,685 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: multiply with args: {"a": 12, "b": 22}
Got output: 264

STARTING TURN 2
---------------



2025-09-05 16:35:25,152 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AgentChatResponse(response='The result of multiplying 12 by 22 is 264.', sources=[ToolOutput(blocks=[TextBlock(block_type='text', text='264')], tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 12, 'b': 22}}, raw_output=264, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [19]:
agent.chat("What is 5 + 2?", tool_choice="add")

2025-09-05 16:35:45,783 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


STARTING TURN 1
---------------



2025-09-05 16:35:46,769 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: add with args: {"a":5,"b":2}
Got output: 7

STARTING TURN 2
---------------



2025-09-05 16:35:48,497 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AgentChatResponse(response='The result of adding 5 and 2 is 7.', sources=[ToolOutput(blocks=[TextBlock(block_type='text', text='7')], tool_name='add', raw_input={'args': (), 'kwargs': {'a': 5, 'b': 2}}, raw_output=7, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)